# Installation of Pytorch
Pytorch binary can be installed by pip, or complied following instructions in https://github.com/pytorch/pytorch#from-source.

Here we will use pip.

## Define a Set of Computation Nodes

| &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Name  &nbsp;&nbsp;&nbsp;&nbsp;  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |   &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Fomula &nbsp;&nbsp;&nbsp;&nbsp;   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |   &nbsp;&nbsp;  Gradients  |
|:-------------:|:------------- |:----- |
| Linear      | $y=x^T\cdot W+b$ | $\frac{\partial \mathcal{L}}{\partial x}=W\cdot\frac{\partial \mathcal{L}}{\partial y}\\\frac{\partial \mathcal{L}}{\partial W}=x^T\cdot\frac{\partial \mathcal{L}}{\partial y}\\\frac{\partial \mathcal{L}}{\partial b}=\frac{\partial \mathcal{L}}{\partial y}$ |
| Sigmoid     | $y=\frac{1}{1+e^{-x}}$  | $\frac{\partial \mathcal{L}}{\partial x}=\frac{\partial \mathcal{L}}{\partial y}(1-y)y$ |
| Softmax     | $y_j=\frac{e^{x_j}}{\sum\limits_i e^{x_i}}$ | $\frac{\partial \mathcal{L}}{\partial x_j}=\frac{\partial \mathcal{L}}{\partial y_j}y_j-y_j\sum\limits_i \frac{\partial \mathcal{L}}{\partial y_i}y_i$ |
| CrossEntropy | $y=-\sum\limits_i p_i \log(x_i)$ | $\frac{\partial \mathcal{L}}{\partial x_i}=-\frac{\partial \mathcal{L}}{\partial y}\frac{p_i}{x_i}$ |
| Mean  | $y=\frac{1}{N}\sum\limits_i x_i$ | $\frac{\partial \mathcal{L}}{\partial x_i}=\frac{1}{N}\frac{\partial \mathcal{L}}{\partial y}$ |

In [1]:
try:
    import torch
except:
    !pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
    import torch

In [2]:
from torchvision import datasets, transforms

# data loader: we split the dataset into training set and test set.
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

Processing...
Done!


In [26]:
import torch.nn as nn
import torch.nn.functional as F

# create a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(28 * 28, 100)
        self.linear2 = nn.Linear(100, 10)

    def forward(self, x):
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        x = x.view(-1, 28*28)
        x = torch.sigmoid(self.linear1(x))
        return F.log_softmax(self.linear2(x), dim = 0)


# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

net = Net().to(device)
print(net)

Net(
  (linear1): Linear(in_features=784, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=10, bias=True)
)


In [29]:
import torch.optim as optim

def train(learning_rate=0.5, epochs=10):

    # create a stochastic gradient descent optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0)
    # create a loss function
    loss_func = nn.NLLLoss()

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            net_out = net(data)
            loss = loss_func(net_out, target)
            loss.backward()
            optimizer.step()
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.data.item()))

    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        net_out = net(data)
        # sum up batch loss
        test_loss += loss_func(net_out, target).data.item()
        # get the index of the max log-probability
        pred = net_out.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:
train()

Train Epoch: 0 	Loss: 3.164179
Train Epoch: 1 	Loss: 3.192139
Train Epoch: 2 	Loss: 3.193585
Train Epoch: 3 	Loss: 3.190815
Train Epoch: 4 	Loss: 3.147254
Train Epoch: 5 	Loss: 3.093015
Train Epoch: 6 	Loss: 3.126472
Train Epoch: 7 	Loss: 3.115194
Train Epoch: 8 	Loss: 3.142654
Train Epoch: 9 	Loss: 3.195892

Test set: Average loss: 0.0158, Accuracy: 9721/10000 (97%)

